In [1]:
# Importing required Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
# Reading the file
import pandas as pd
twitter= pd.read_csv('/content/Twitter_comments_class.csv')
twitter.columns= ['SrNo', 'tag', 'Sentiment', 'Tweet']

In [3]:
twitter

,SrNo,tag,Sentiment,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [4]:
twitter.isnull().sum()  # checking the null values

,0
SrNo,0
tag,0
Sentiment,0
Tweet,858


In [5]:
twitter=twitter.dropna(axis='rows')  # dropping null values

In [6]:
twitter.shape

(73824, 4)

In [7]:
twitter.Sentiment.value_counts()   # checking the counts of each sentiment

,count
Sentiment,
Negative,22312
Positive,20619
Neutral,18051
Irrelevant,12842


In [8]:
val=twitter.Sentiment
val

,Sentiment
0,Positive
1,Positive
2,Positive
3,Positive
4,Positive
...,...
74677,Positive
74678,Positive
74679,Positive
74680,Positive


In [9]:
from sklearn.preprocessing import LabelEncoder   # Encoding the sentiments
le=LabelEncoder()
labels= le.fit_transform(val)
labels

array([3, 3, 3, ..., 3, 3, 3])

In [10]:
sentences= twitter.Tweet
sentences[0]

'im getting on borderlands and i will murder you all ,'

In [11]:
# Importing required libraries
import re  # regular expressions
import nltk  # natural language tool kit
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [12]:
stop= set(stopwords.words('english'))
stem= PorterStemmer()
lemma= WordNetLemmatizer()

In [13]:
# Cleaning the text
def text_cleaner(text):
    text= str(text).lower()
    text= re.sub(r'\d+', '', text)
    text= re.sub(r'[^\w\s]', '', text)

    text= ' '.join([word for word in text.split() if word not in stop])
    text= " ".join(stem.stem(word) for word in text.split())
    # text= " ".join(lemma.lemmatize(word) for word in text.split())
    return text

In [14]:
# Adding a new column of cleaned comments
twitter['Comments_cleaned']= twitter['Tweet'].apply(text_cleaner)
twitter

<ipython-input-14-fab9bd0750f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter['Comments_cleaned']= twitter['Tweet'].apply(text_cleaner)


,SrNo,tag,Sentiment,Tweet,Comments_cleaned
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im get borderland murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im get borderland kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im come borderland murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im get borderland murder
...,...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,realiz window partit mac like year behind nvid...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,realiz mac window partit year behind nvidia dr...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,realiz window partit mac year behind nvidia dr...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,realiz window partit mac like year behind nvid...


In [15]:
sentences= twitter.Comments_cleaned
sentences[0]

'im get borderland murder'

In [16]:
# Tokenize the sentences
tokenizer = Tokenizer()
clean_sentences = [sentence for sentence in sentences ]
len(clean_sentences)

73824

In [17]:
tokenizer.fit_on_texts(clean_sentences)
word_index = tokenizer.word_index

In [18]:
word_index

{'game': 1,
 'play': 2,
 'get': 3,
 'like': 4,
 'im': 5,
 'fuck': 6,
 'love': 7,
 'go': 8,
 'one': 9,
 'good': 10,
 'time': 11,
 'realli': 12,
 'new': 13,
 'see': 14,
 'look': 15,
 'make': 16,
 'peopl': 17,
 'johnson': 18,
 'shit': 19,
 'thank': 20,
 'cant': 21,
 'dont': 22,
 'best': 23,
 'still': 24,
 'player': 25,
 'year': 26,
 'even': 27,
 'got': 28,
 'day': 29,
 'great': 30,
 'work': 31,
 'xbox': 32,
 'want': 33,
 'googl': 34,
 'pleas': 35,
 'facebook': 36,
 'amazon': 37,
 'dead': 38,
 'rhandlerr': 39,
 'know': 40,
 'call': 41,
 'verizon': 42,
 'back': 43,
 'microsoft': 44,
 'say': 45,
 'us': 46,
 'much': 47,
 'ps': 48,
 'ban': 49,
 'use': 50,
 'fifa': 51,
 'red': 52,
 'come': 53,
 'guy': 54,
 'would': 55,
 'tri': 56,
 'first': 57,
 'fix': 58,
 'ive': 59,
 'wait': 60,
 'watch': 61,
 'today': 62,
 'home': 63,
 'video': 64,
 'pubg': 65,
 'think': 66,
 'nvidia': 67,
 'fun': 68,
 'borderland': 69,
 'live': 70,
 'gta': 71,
 'also': 72,
 'team': 73,
 'unk': 74,
 'thing': 75,
 'need': 76,

In [19]:
# Convert sentences to sequences of indices
sequences = tokenizer.texts_to_sequences(clean_sentences)

# Pad sequences to ensure they have the same length
maxlen = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

padded_sequences.shape

(73824, 163)

In [20]:
labels.shape

(73824,)

In [21]:
np.unique(labels)

array([0, 1, 2, 3])

# Defining RNN model

In [22]:
# RNN model
embedding_dim = 16
vocab_size = len(word_index) + 1

# Define RNN model for multi-class classification
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    SimpleRNN(32),  # Simple RNN layer with 32 units
    Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, labels, epochs=10, verbose=1)

# Test the model
test_sentence = 'This movie is amazing'
test_sequence = tokenizer.texts_to_sequences([test_sentence])
padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
prediction = model.predict(padded_test_sequence)
predicted_class = np.argmax(prediction)
print("Sentiment prediction:", predicted_class)

Epoch 1/10
2307/2307 [==============================] - 67s 28ms/step - loss: 0.9084 - accuracy: 0.6291
Epoch 2/10
2307/2307 [==============================] - 66s 29ms/step - loss: 0.3793 - accuracy: 0.8634
Epoch 3/10
2307/2307 [==============================] - 63s 27ms/step - loss: 0.2407 - accuracy: 0.9108
Epoch 4/10
2307/2307 [==============================] - 62s 27ms/step - loss: 0.1902 - accuracy: 0.9298
Epoch 5/10
2307/2307 [==============================] - 61s 27ms/step - loss: 0.1537 - accuracy: 0.9425
Epoch 6/10
2307/2307 [==============================] - 63s 27ms/step - loss: 0.1451 - accuracy: 0.9453
Epoch 7/10
2307/2307 [==============================] - 64s 28ms/step - loss: 0.1416 - accuracy: 0.9459
Epoch 8/10
2307/2307 [==============================] - 63s 27ms/step - loss: 0.1510 - accuracy: 0.9434
Epoch 9/10
2307/2307 [==============================] - 63s 27ms/step - loss: 0.1601 - accuracy: 0.9384
Epoch 10/10
1/1 [==============================] - 0s 176ms/step

In [23]:
# Applying gradio interface
!pip install gradio
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Define the prediction function
def predict_sentiment(text):
    # Preprocess the input text
    test_sequence = tokenizer.texts_to_sequences([text])
    padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
    # Predict sentiment using the model
    prediction = model.predict(padded_test_sequence)
    predicted_class = np.argmax(prediction)

    # Use LabelEncoder to map predicted class index to original sentiment label
    predicted_label = le.inverse_transform([predicted_class])[0]

    return predicted_label

# Create a Gradio interface
iface = gr.Interface(fn=predict_sentiment,
                     inputs="text",
                     outputs="text",
                     title="Sentiment Analysis",
                     description="Enter a text to predict its sentiment (negative, somewhat negative, somewhat positive, or positive).")

# Launch the interface
iface.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.1 MB/s eta 0:00:00
Setting queue=True in a Colab notebook requires shari

# Defining LSTM model

In [25]:
from tensorflow.keras.layers import LSTM, Dropout

# Define a more complex LSTM model for multi-class classification
model_complex_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    LSTM(64, return_sequences=True),  # First LSTM layer with 64 units and return sequences
    Dropout(0.5),  # Dropout layer to reduce overfitting
    LSTM(32),  # Second LSTM layer with 32 units
    Dropout(0.5),  # Dropout layer
    Dense(64, activation='relu'),  # Dense layer with ReLU activation
    Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

In [26]:
model_complex_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model_complex_lstm.fit(padded_sequences, labels, epochs=10, verbose=1)

Epoch 1/10
2307/2307 [==============================] - 224s 96ms/step - loss: 0.9761 - accuracy: 0.5940
Epoch 2/10
2307/2307 [==============================] - 220s 95ms/step - loss: 0.5150 - accuracy: 0.8164
Epoch 3/10
2307/2307 [==============================] - 221s 96ms/step - loss: 0.3616 - accuracy: 0.8735
Epoch 4/10
2307/2307 [==============================] - 221s 96ms/step - loss: 0.2850 - accuracy: 0.8980
Epoch 5/10
2307/2307 [==============================] - 222s 96ms/step - loss: 0.2482 - accuracy: 0.9101
Epoch 6/10
2307/2307 [==============================] - 222s 96ms/step - loss: 0.2151 - accuracy: 0.9219
Epoch 7/10
2307/2307 [==============================] - 219s 95ms/step - loss: 0.1952 - accuracy: 0.9291
Epoch 8/10
2307/2307 [==============================] - 221s 96ms/step - loss: 0.1772 - accuracy: 0.9343
Epoch 9/10
2307/2307 [==============================] - 220s 96ms/step - loss: 0.1639 - accuracy: 0.9392
Epoch 10/10
2307/2307 [==============================] 

In [28]:
# Test the complex LSTM model
test_sentence = 'This movie is amazing'
test_sequence = tokenizer.texts_to_sequences([test_sentence])
padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
prediction_complex_lstm = model_complex_lstm.predict(padded_test_sequence)
predicted_class_complex_lstm = np.argmax(prediction_complex_lstm)
print("Sentiment prediction using complex LSTM model:", predicted_class_complex_lstm)

1/1 [==============================] - 1s 1s/step
Sentiment prediction using complex LSTM model: 3


In [29]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Define the prediction function
def predict_sentiment(text):
    # Preprocess the input text
    test_sequence = tokenizer.texts_to_sequences([text])
    padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
    # Predict sentiment using the model
    prediction = model_complex_lstm.predict(padded_test_sequence)
    predicted_class = np.argmax(prediction)

    # Use LabelEncoder to map predicted class index to original sentiment label
    predicted_label = le.inverse_transform([predicted_class])[0]

    return predicted_label

# Create a Gradio interface
iface = gr.Interface(fn=predict_sentiment,
                     inputs="text",
                     outputs="text",
                     title="Sentiment Analysis",
                     description="Enter a text to predict its sentiment (negative, somewhat negative, somewhat positive, or positive).")

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2a49ea8a7eb06e78f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
